In [1]:
from sklearn.datasets import load_iris  
import numpy as np
from IrisFisher import softmax, softmax_grad, logloss, logloss_grad

In [2]:
data = load_iris()

def UnisonShuffle(a, b):
    ''' Функция совместного перемешивания'''
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def random_init(size, left = -1., right = 1.):
    return np.random.random(size = size) * (right + left) + left

def net_init(params):
    params["L0"] = np.zeros((1, 4))
    params["W0"] = random_init((4, 3))
    params['b0'] = random_init((1, 3))
    params["L1"] = np.zeros((1, 3))
    params["P1"] = np.zeros((1, 3))
    params["E"] = np.zeros((1,1))
    

In [ ]:
def forward_prop_1(X, y, params):
    if X.shape !=  (1, 4):
        raise ValueError("X : Expect shape (1, 4), given {0}".format(X.shape))
    if y.shape !=  (1, 3):
        raise ValueError("X : Expect shape (1, 3), given {0}".format(y.shape))
    
    
    params["L0"] = X
    params["L1"] = np.dot(params["L0"], params["W0"]) + params["b0"]
    params["P1"] = softmax(params["L1"])
    
def back_prop_1(X, y, params, giper_params):
    pass
    

In [38]:
X = data["data"]
y_lables = data["target"]
y = np.zeros((len(y_lables), 3))
for i, a in enumerate(y_lables):
    y[i][a] = 1

X, y = UnisonShuffle(X, y)
X = X.reshape(-1, 1, 4)
y = y.reshape(-1, 1, 3)
params = dict()
giper_params = dict()
giper_params["learning rate"] = 0.1
lenX = len(y)


net_init(params)
    

(150, 4)
(150, 1, 4)


In [33]:
for i in range(1000):
    pass
    # forward_prop(X[i%lenX], y[i%lenX], params)
    # back_prop(X[i%lenX], y[i%lenX], params, giper_params)

(3, 1)
[[ 0.42978736]
 [ 0.00718448]
 [-0.43697184]] [[6.4 2.8 5.6 2.2]]
--------------------------------------------------------------
[[ 2.75063909  1.2034046   2.40680921  0.94553219]
 [ 0.04598067  0.02011654  0.04023309  0.01580586]
 [-2.79661976 -1.22352115 -2.44704229 -0.96133804]]
[[ 0.32505199 -0.99981285  0.05235096]
 [-0.69387893  0.93410819  0.54137767]
 [-0.49760059 -0.31555789 -0.49513333]
 [ 0.43595052 -0.1049747  -0.22641742]]


ValueError: operands could not be broadcast together with shapes (4,3) (3,4) (4,3) 